# Bài toán được áp dụng mô hình học máy:

1. Dự đoán giá trị đơn hàng (Purchase Amount (USD)) dựa trên các đặc trưng.
2. 
---

## Nhập thư viện:

In [1]:
!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Đọc dữ liệu:

In [3]:
df = pd.read_csv('../Datasets/data_cleaned.csv')
df.sample(5)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Previous Purchases,Payment Method,Frequency of Purchases
1931,1932,54,Male,Jewelry,Accessories,42,Illinois,S,Olive,Spring,2.7,No,Store Pickup,No,6,Debit Card,Every 3 Months
1232,1233,57,Male,Handbag,Accessories,23,New York,M,Charcoal,Summer,3.6,No,Free Shipping,Yes,31,Debit Card,Every 3 Months
406,407,52,Male,Gloves,Accessories,55,Maryland,M,Peach,Spring,4.6,Yes,Express,Yes,13,Cash,Monthly
1138,1139,18,Male,Pants,Clothing,36,Iowa,M,Purple,Summer,4.7,No,Express,Yes,32,Venmo,Fortnightly
359,360,31,Male,Sneakers,Footwear,74,Georgia,M,Olive,Winter,3.0,Yes,Store Pickup,Yes,30,Credit Card,Every 3 Months


## Tiền xử lý dữ liệu trước khi áp dụng mô hình học máy:

### Loại bỏ cột `Customer ID` vì nó chỉ có vai trò định danh, không ảnh hưởng đến mô hình học máy.

In [4]:
df = df.drop('Customer ID', axis=1)

### Loại bỏ cột `Size` vì tác giả sử dụng hệ thống kích thước chung cho tất cả các mặt hàng không liên quan (Quần áo, Giày dép, áo khoác, phụ kiện). Đây chính là thuộc tính gây nhiễu cho mô hình.

In [5]:
df = df.drop('Size', axis=1)

## Bài toán 1: Dự đoán giá trị đơn hàng (Purchase Amount (USD)) dựa trên các đặc trưng.

### Tách và chuẩn hóa dữ liệu:

#### Tách dữ liệu thành tập nguồn (X) và tập đích (Y)

In [6]:
X1= df.drop('Purchase Amount (USD)', axis=1)
y1 = df['Purchase Amount (USD)']

In [7]:
X1.sample(3)

,Age,Gender,Item Purchased,Category,Location,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Previous Purchases,Payment Method,Frequency of Purchases
2192,41,Male,Scarf,Accessories,Virginia,Yellow,Winter,3.2,No,Next Day Air,No,9,Credit Card,Monthly
1553,27,Male,Blouse,Clothing,Delaware,Indigo,Spring,4.8,No,Free Shipping,Yes,21,Debit Card,Quarterly
1722,46,Male,T-shirt,Clothing,Massachusetts,Violet,Summer,3.3,No,Standard,No,14,Cash,Fortnightly


In [8]:
y1.sample(3)

2949    93
215     64
205     64
Name: Purchase Amount (USD), dtype: int64

#### Phân loại các cột theo kiểu dữ liệu:

In [9]:
num_cols = X1.select_dtypes(include=np.number).columns.to_list()
cat_cols = X1.select_dtypes(exclude=np.number).columns.to_list()

In [10]:
num_cols

['Age', 'Review Rating', 'Previous Purchases']

In [11]:
cat_cols

['Gender',
 'Item Purchased',
 'Category',
 'Location',
 'Color',
 'Season',
 'Subscription Status',
 'Shipping Type',
 'Discount Applied',
 'Payment Method',
 'Frequency of Purchases']

#### Tạo transformer cho các cột số sử dụng StandardScaler:

In [12]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

#### Tạo transformer cho các cột phân loại sử dụng OneHotEncoder:

In [13]:
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

#### Kết hợp các transformer lại với nhau qua ColumnTransformer:

In [14]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_cols),
                                               ('cat', categorical_transformer, cat_cols)])

#### Tách dữ liệu thành tập huấn luyện và kiểm tra:

In [15]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

#### Fit trên X_train và transform cho cả train và test:

In [16]:
X1_train_processed = preprocessor.fit_transform(X1_train)
X1_test_processed = preprocessor.transform(X1_test)

### Lựa chọn mô hình:

Sử dụng mô hình `Linear Regression`. Lý do:
- Đơn giản và hiệu quả: `Linear Regression` là một mô hình cơ bản, dễ hiểu và thực thi nhanh, phù hợp với các bài toán hồi quy.
- Giải thích kết quả: Mô hình tuyến tính giúp dễ dàng diễn giải tầm quan trọng của các đặc trưng đối với giá trị dự đoán.
- Hiệu quả trên dữ liệu đã được tiền xử lý: Sau khi dữ liệu được chuẩn hóa, mô hình tuyến tính thường cho kết quả ổn định.

### Cài đặt mô hình:

#### Huấn luyện mô hình:

In [20]:
model1 = TransformedTargetRegressor(
    regressor=LinearRegression(),
    transformer=StandardScaler()
)

In [21]:
model1.fit(X1_train_processed, y1_train)

TransformedTargetRegressor(regressor=LinearRegression(),
                           transformer=StandardScaler())

#### Dự đoán trên tập test:

In [ ]:
y1_pred = model1.predict(X1_test_processed)

### Đánh giá mô hình:

#### Tính các chỉ số đánh giá mô hình:

In [23]:
# MSE: Trung bình bình phương sai số, phạt mạnh các sai số lớn.
mse = mean_squared_error(y1_test, y1_pred)
print('Mean Squared Error:', mse)

# MAE: Trung bình sai số tuyệt đối, cho biết mức độ lệch trung bình giữa giá trị dự đoán và thực tế.
mae = mean_absolute_error(y1_test, y1_pred)
print('Mean Absolute Error:', mae)

# RMSE: Căn bậc hai của MSE, giúp đưa lỗi về cùng đơn vị với target.
rmse = np.sqrt(mse)
print('Root Mean Squared Error:', rmse)

# R²: Hệ số xác định, cho biết phần trăm biến thiên của target được giải thích bởi mô hình.
r2 = r2_score(y1_test, y1_pred)
print('R²:', r2)

Mean Squared Error: 575.9932944735009
Mean Absolute Error: 20.901952800459753
Root Mean Squared Error: 23.99986030112469
R²: -0.02932709017618018


#### Đánh giá:

- MSE (575.99) và RMSE (khoảng 24):
    - Giá trị trung bình của sai số dự đoán là khoảng 24 USD. Điều này cho thấy mức độ chênh lệch giữa giá trị dự đoán và giá trị thực tế khá cao, so với phạm vi giá trị của target từ khoảng 20-100 USD.
- MAE (khoảng 20.90):
    - Sai số tuyệt đối trung bình khoảng 21 USD tức trung bình mỗi dự đoán lệch khoảng 21 USD so với giá trị thực.
- R² (-0.0293):
    - Hệ số xác định âm cho thấy mô hình hiện tại không chỉ không giải thích được biến thiên của dữ liệu, R² âm thường chỉ ra rằng mô hình chưa được xây dựng phù hợp hoặc có thể có hiện tượng underfitting.

---

## Bài toán 2: 